In [5]:
import pandas as pd

# Não sei como vai funcionar essa parte.
arquivo = "/app/src/notebooks/???/GEAP.csv"
GEAP_df = pd.read_csv(arquivo, sep=';')
GEAP_df.columns = GEAP_df.columns.str.strip()


In [6]:
class Pessoas:
    def __init__(self, tipo_registro, cpf, nome, conveniada, data_nascimento, tipo_vinculo, plano, ref_cobranca, dta_inscricao, dta_movimentacao, tipo_custeio, vlr_integral, vlr_remuneracao, vlr_contribuicao, vlr_percapita, vlr_participacao) -> None:
        self.tipo_registro = tipo_registro
        self.cpf = cpf
        self.nome = nome
        self.conveniada = conveniada
        self.data_nascimento = data_nascimento
        self.tipo_vinculo = tipo_vinculo
        self.plano = plano
        self.ref_cobranca = ref_cobranca
        self.dta_inscricao = dta_inscricao
        self.dta_movimentacao = dta_movimentacao
        self.tipo_custeio = tipo_custeio
        self.vlr_integral = vlr_integral
        self.vlr_remuneracao = vlr_remuneracao
        self.vlr_contribuicao = vlr_contribuicao
        self.vlr_percapita = vlr_percapita
        self.vlr_participacao = vlr_participacao
        
class Servidores:
    def __init__(self, pessoa: Pessoas, cod_matricula, nroinscricao) -> None:
        self.pessoa = pessoa
        self.cod_matricula = cod_matricula
        self.nroinscricao = nroinscricao
        self.dependentes = []

    def add_dependente(self, dependente):
        if dependente not in self.dependentes:
            self.dependentes.append(dependente)
            dependente.titular = self
        else:
            print(f"Dependente {dependente.pessoa.nome} já está vinculado a este titular.")

    def rem_dependente(self, dependente):
        if dependente in self.dependentes:
            self.dependentes.remove(dependente)
            dependente.titular = None
        else:
            print(f"Dependente {dependente.pessoa.nome} não está vinculado a este titular.")


class Dependentes:
    def __init__(self, pessoa: Pessoas, nroinscricao) -> None:
        self.pessoa = pessoa
        self.nroinscricao = nroinscricao
        self.titular = None

    def get_titular(self):
        if self.titular:
            return f"Titular: {self.titular.pessoa.nome}"
        else:
            return "Este dependente não possui um titular associado."

In [7]:
# Elaboração dos dados a partir das classes.

titulares: list[Servidores] = []
dependentes: list[Dependentes] = []

for i, row in GEAP_df.iterrows():
    pessoa = Pessoas(
            tipo_registro=row['0-TipoRegistro'],
            cpf=row['CPF'],
            nome=row['NmeCliente'],
            conveniada=row['Conveniada'],
            data_nascimento=row['DtaNascimento'],
            tipo_vinculo=row['TipoVinculo'],
            plano=row['Plano'],
            ref_cobranca=row['RefCobranca'],
            dta_inscricao=row['DtaInscricao'],
            dta_movimentacao=row['DtaMovimentacao'],
            tipo_custeio=row['TipoCusteio'],
            vlr_integral=row['VlrIntegral'],
            vlr_remuneracao=row['VlrRemuneracao'],
            vlr_contribuicao=row['VlrContribuicao'],
            vlr_percapita=row['VlrPerCapita'],
            vlr_participacao=row['VlrParticipacao']
    )
    if row['TipoVinculo'] == "TITULAR":
        servidor = Servidores(pessoa, cod_matricula=row['CodMatricula'], nroinscricao=row['NroInscricao'])
        titulares.append(servidor)
    else:
        dependente = Dependentes(pessoa, nroinscricao=row['NroInscricao'])
        dependentes.append(dependente)

# Relaciona os Dependentes aos seus titulares
servidores_por_inscricao = {servidor.nroinscricao: servidor for servidor in titulares}

for dependente in dependentes:
    servidor = servidores_por_inscricao.get(dependente.nroinscricao)
    
    if servidor:
        servidor.add_dependente(dependente)
    else:
        print(f"Erro: Não foi encontrado um servidor com a inscrição {dependente.nroinscricaoa} para o dependente {dependente.pessoa.nome}.")


In [11]:
# Novo DataFrame a partir das classes/listas
# Esta célula é apenas para visualização dos dados das classes.
dados = []

for servidor in titulares:
    seq_cliente = 0

    dados.append({
        '0-TipoRegistro': servidor.pessoa.tipo_registro,
        'CodMatricula': servidor.cod_matricula,
        'CPF': servidor.pessoa.cpf,
        'NmeCliente': servidor.pessoa.nome,
        'Conveniada': servidor.pessoa.conveniada,
        'DtaNascimento': servidor.pessoa.data_nascimento,
        'NroInscricao': servidor.nroinscricao,
        'SeqCliente': seq_cliente,
        'TipoVinculo': servidor.pessoa.tipo_vinculo,
        'Plano': servidor.pessoa.plano,
        'RefCobranca': servidor.pessoa.ref_cobranca,
        'DtaInscricao': servidor.pessoa.dta_inscricao,
        'DtaMovimentacao': servidor.pessoa.dta_movimentacao,
        'TipoCusteio': servidor.pessoa.tipo_custeio,
        'VlrIntegral': servidor.pessoa.vlr_integral,
        'VlrRemuneracao': servidor.pessoa.vlr_remuneracao,
        'VlrContribuicao': servidor.pessoa.vlr_contribuicao,
        'VlrPerCapita': servidor.pessoa.vlr_percapita,
        'VlrParticipacao': servidor.pessoa.vlr_participacao
    })
    
    for i, dependente in enumerate(servidor.dependentes, start=1):
        dados.append({
        '0-TipoRegistro': dependente.pessoa.tipo_registro,
        'CodMatricula': servidor.cod_matricula,
        'CPF': dependente.pessoa.cpf,
        'NmeCliente': dependente.pessoa.nome,
        'Conveniada': dependente.pessoa.conveniada,
        'DtaNascimento': dependente.pessoa.data_nascimento,
        'NroInscricao': dependente.nroinscricao,
        'SeqCliente': i,
        'TipoVinculo': dependente.pessoa.tipo_vinculo,
        'Plano': dependente.pessoa.plano,
        'RefCobranca': dependente.pessoa.ref_cobranca,
        'DtaInscricao': dependente.pessoa.dta_inscricao,
        'DtaMovimentacao': dependente.pessoa.dta_movimentacao,
        'TipoCusteio': dependente.pessoa.tipo_custeio,
        'VlrIntegral': dependente.pessoa.vlr_integral,
        'VlrRemuneracao': dependente.pessoa.vlr_remuneracao,
        'VlrContribuicao': dependente.pessoa.vlr_contribuicao,
        'VlrPerCapita': dependente.pessoa.vlr_percapita,
        'VlrParticipacao': dependente.pessoa.vlr_participacao
        })


GEAP_dfFinal = pd.DataFrame(dados)

# Fim do transporte dos dados da planilha para as classes. Caso queira visualizar a planilha:
# display(GEAP_dfFinal)

,0-TipoRegistro,CodMatricula,CPF,NmeCliente,Conveniada,DtaNascimento,NroInscricao,SeqCliente,TipoVinculo,Plano,RefCobranca,DtaInscricao,DtaMovimentacao,TipoCusteio,VlrIntegral,VlrRemuneracao,VlrContribuicao,VlrPerCapita,VlrParticipacao
0,1,34060418,010.891.014-81,ADILMA CRISTINA DANTAS DE MELO,DPE/RR,12/07/1980,1301642,0,TITULAR,GEAPSAUDE VIDA,jul/24,12/04/2023,NaN,COPATROCINADO,728.89,10059.79,646.01,0.0,0.0
1,1,34060418,151.894.614-32,ANA EVELYM DE MELO RODRIGUES,DPE/RR,26/01/2006,1301642,1,FILHO(A),GEAPSAUDE VIDA,jul/24,28/11/2023,NaN,COPATROCINADO,359.27,10059.79,318.42,0.0,0.0
2,1,367010422,029.704.952-60,ADRIA LETICIA BRAGA SOMBRA,DPE/RR,27/06/1998,1297697,0,TITULAR,GEAPSAUDE VIDA,jul/24,01/03/2023,NaN,COPATROCINADO,475.12,5000.00,421.09,0.0,0.0
3,1,367010422,021.443.702-73,IAGO CARVALHO DE SALES,DPE/RR,29/10/1998,1297697,1,COMPANHEIRO(A),GEAPSAUDE VIDA,jul/24,27/04/2023,NaN,COPATROCINADO,475.12,5000.00,421.09,0.0,0.0
4,1,79070611,447.338.952-91,ADRIANA PATRICIA FARIAS DE LIMA,DPE/RR,09/06/1977,1297734,0,TITULAR,GEAPSAUDE VIDA,jul/24,01/03/2023,NaN,COPATROCINADO,881.95,5000.00,781.66,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,1,10310702,052.988.642-10,LUIZA PANTOJA LEITE,DPE/RR,08/08/2015,1297717,4,FILHO(A),GEAPSAUDE VIDA,jul/24,01/03/2023,NaN,COPATROCINADO,359.27,5000.00,318.42,0.0,0.0
615,1,431010223,006.468.042-86,YAGO DJANGO SOUZA DE OLIVEIRA,DPE/RR,23/04/1992,1299647,0,TITULAR,GEAPSAUDE VIDA,jul/24,23/03/2023,NaN,COPATROCINADO,546.39,9359.52,484.26,0.0,0.0
616,1,431010223,025.726.612-76,ELLEN ALVES RUFINO,DPE/RR,12/08/1995,1299647,1,CONJUGE,GEAPSAUDE VIDA,jul/24,23/03/2023,NaN,COPATROCINADO,475.12,9359.52,421.09,0.0,0.0
617,1,272011018,020.758.892-95,YNAIMA SUANAM RIOS SANTANA,DPE/RR,10/06/1993,1297684,0,TITULAR,GEAPSAUDE VIDA,jul/24,01/03/2023,NaN,COPATROCINADO,546.39,5000.00,484.26,0.0,0.0
